In [1]:
#!pulptest
!pip install pyomo
#!apt install glpk-utils
#!pip install glpk
#!glpsol --version
!pip install easydev
!pip install colormap


     |████████████████████████████████| 11.0 MB 7.9 MB/s eta 0:00:01


In [2]:
!pip install gurobipy


     |████████████████████████████████| 12.8 MB 1.3 MB/s eta 0:00:01


In [1]:
import numpy as np
from gurobipy import *
import random
from colormap import rgb2hex
import pandas as pd
import networkx as nx

In [6]:
nums_invalid/nums_total

0.265

In [6]:
#pathway_list = ['Purine', 'Carbon metabolism', 'Nucleotide metabolism', 'Biosynthesis of amino acids', 'Biosynthesis of cofactors']
pathway_list = ['Glycerolipid', 'Citrate_cycle', 'Pentose_phosphate', 'Pyruvate', 'Galactose']
#pathway_list = ['Glycerolipid', 'Citrate_cycle', 'Pentose_phosphate', 'Glycine_serine_threonine', 'Galactose']

#pathway_list = ['Glycerolipid']


#species_list = ['eco']
species_list = ['hsa', 'mmu']

data_type = 'small'
import networkx as nx

nums_invalid = 0
nums_total = 0

for species_name  in species_list:
    for pathway_name in pathway_list:
        print(pathway_name)
        list_enzymes = []
        list_reactions = []
        list_compounds = []
        nodes = []
        edges = []
        full_edges = []
        enzy_dict = {}
        mode = 0
        num_edges = 0
        with open(data_type+'_data/'+species_name+'_filtering/'+pathway_name+'.input') as f:
            while mode <= 3:
                if mode < 3:
                    num_nodes = int(f.readline())
                      #print(num_nodes)
                    for i in range(0, num_nodes):
                        element = f.readline()
                        #print(element)
                        nodes.append(element[0:-1])
                        if mode == 0:
                            list_enzymes.append(element[0:-1])
                        elif mode == 1:
                            list_reactions.append(element[0:-1])
                        else:
                            list_compounds.append(element[0:-1])
                    mode = mode + 1
                else:
                    num_edges = int(f.readline())
                    for i in range(0, num_edges):
                        line = f.readline()
                        element = line[0:-1].split(' ')
                        full_edges.append(element)
                        if element[0] not in list_enzymes and element[1] not in list_enzymes:
                            edges.append(element)
                        else:
                            if element[1] not in enzy_dict.keys():
                                enzy_dict[element[1]] = [element[0]]
                            else:
                                enzy_dict[element[1]].append(element[0])
                    mode = mode + 1
        
        G = nx.DiGraph()
        G_metabolic = nx.DiGraph()
        G.add_edges_from(edges)
        G_metabolic.add_edges_from(full_edges)

        idx_reac = 0
        idx_comp = 0
        for node in G.nodes:
            if node in list_reactions:
                G.nodes[node]['index'] = idx_reac
                idx_reac = idx_reac + 1
            else:
                G.nodes[node]['index'] = idx_comp
                idx_comp = idx_comp + 1    
    
        testcase = []
        with open(data_type+'_output/'+species_name+'/'+pathway_name+'.out') as f:
            while True:
                record = {}
                line = f.readline()
                if line == '':
                    break
                line = f.readline()
                record['Target_size'] = int(line)

                line = f.readline()
                target = []
                for i in range(0, record['Target_size']):
                    line = f.readline()
                    target.append(line[0:-1])
                record['Target'] = target

                line = f.readline()
                line = f.readline()
                record['DI_damage'] = int(line)

                #if pathway_name != 'Purine':
                line = f.readline()
                line = f.readline()
                record['DI_running_time'] = float(line)

                line = f.readline()
                line = f.readline()
                record['QC_damage'] = float(line)

                line = f.readline()
                line = f.readline()
                record['QC_QPU_access_time'] = float(line)

                line = f.readline()
                line = f.readline()
                record['QC_running_time'] = float(line)
                testcase.append(record)
        #print(testcase)
        countt = 0
        for record in testcase:
            #if i!=2:
            #    i = i+1
            #    continue
            #i = i + 1
            #print(list_compounds)
            #print(record['Target'])
            target_list = record['Target']
            res_IP, runtime_IP, solution = gurobi_solve(list_reactions, list_compounds, target_list, edges, G)
            
            selection_enzy = []
            for reac in list_reactions:
                key = 'FE[0,'+str(G.nodes[reac]['index']) + ']'
                if solution[key] == 1:
                    selection_enzy.append(enzy_dict[reac])
            
            sample_times = 10
            avg_damage = 0
            for t in range(sample_times):
                selected_enzymes = []
                for lst in selection_enzy:
                    selected_enzymes.append(random.choice(lst))
                state = [0]*len(list_enzymes)
                for i in range(0, len(list_enzymes)):
                    if list_enzymes[i] in selected_enzymes:
                        state[i] = 1
                damage, check = bfs(state, target_list, G_metabolic)
                avg_damage = avg_damage + damage
            avg_damage = avg_damage / sample_times
            if res_IP != record['DI_damage']:
                print("FAULT: ",avg_damage, res_IP, record['DI_damage'])
                nums_invalid = nums_invalid + 1
            nums_total = nums_total + 1
            #continue
            if countt == 0:
                f = open(data_type+"_output/"+species_name+"/IP_Res/"+pathway_name+".out", "w")
                countt = countt + 1
            else:
                f = open(data_type+"_output/"+species_name+"/IP_Res/"+pathway_name+".out", "a")
            f.write('Target_size\n')
            f.write(str(record['Target_size'])+'\n')
            f.write('Target\n')
            for enzy in record['Target']:
                f.write(enzy+'\n')
            f.write('Exact_damage\n')
            f.write(str(record['DI_damage'])+'\n')
            f.write('Exact_running_time\n')
            f.write(str(record['DI_running_time'])+'\n')
            f.write('QC_damage\n')
            f.write(str(record['QC_damage'])+'\n')
            f.write('QC_QPU_access_time\n')
            f.write(str(record['QC_QPU_access_time'])+'\n')
            f.write('QC_running_time\n')
            f.write(str(record['QC_running_time'])+'\n')
            f.write('IP_damage\n')
            f.write(str(avg_damage)+'\n')
            f.write('IP_running_time\n')
            f.write(str(runtime_IP)+'\n')
            f.close()

Glycerolipid
IP Starts
Set parameter TimeLimit to value 30
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU E5-2697 v4 @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 36 physical cores, 72 logical processors, using up to 32 threads

Optimize a model with 12193 rows, 7546 columns and 26709 nonzeros
Model fingerprint: 0xb064bf90
Variable types: 0 continuous, 7546 integer (7546 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 8191 rows and 6315 columns
Presolve time: 0.24s
Presolved: 4002 rows, 1231 columns, 9585 nonzeros
Variable types: 0 continuous, 1231 integer (1231 binary)
Found heuristic solution: objective 5.0000000
Found heuristic solution: objective 1.0000000

Explored 0 nodes (0 simplex iterations) in 0.26 seconds (0.17 work units)
Thread count was 32 (of 72 available processors)

Solution coun

  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 12051 rows and 7494 columns
Presolve time: 0.15s
Presolved: 147 rows, 52 columns, 342 nonzeros
Variable types: 0 continuous, 52 integer (52 binary)
Found heuristic solution: objective 3.0000000

Explored 0 nodes (0 simplex iterations) in 0.16 seconds (0.12 work units)
Thread count was 32 (of 72 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
IP Starts
Set parameter TimeLimit to value 30
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU E5-2697 v4 @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 36 physical cores, 72 logical processors, using up to 32 threads

Optimize a model with 12198 rows, 7546 columns and 26714 nonzeros
Model fingerprint: 0xd7a8d882
Variable types: 0 continuous, 7546 integer (7546 binary)
Coefficient statistics:
  M

Thread count: 36 physical cores, 72 logical processors, using up to 32 threads

Optimize a model with 29325 rows, 15860 columns and 66993 nonzeros
Model fingerprint: 0xff5fa6cc
Variable types: 0 continuous, 15860 integer (15860 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 28723 rows and 15734 columns
Presolve time: 1.38s
Presolved: 602 rows, 126 columns, 1568 nonzeros
Variable types: 0 continuous, 126 integer (126 binary)
Found heuristic solution: objective 5.0000000

Explored 0 nodes (0 simplex iterations) in 1.40 seconds (0.78 work units)
Thread count was 32 (of 72 available processors)

Solution count 1: 5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%
IP Starts
Set parameter TimeLimit to value 30
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: In

IP Starts
Set parameter TimeLimit to value 30
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU E5-2697 v4 @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 36 physical cores, 72 logical processors, using up to 32 threads

Optimize a model with 29330 rows, 15860 columns and 66998 nonzeros
Model fingerprint: 0x097e1113
Variable types: 0 continuous, 15860 integer (15860 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 29310 rows and 15851 columns
Presolve time: 0.45s
Presolved: 20 rows, 9 columns, 44 nonzeros
Variable types: 0 continuous, 9 integer (9 binary)
Found heuristic solution: objective 5.0000000

Explored 0 nodes (0 simplex iterations) in 0.47 seconds (0.35 work units)
Thread count was 32 (of 72 available processors)

Solution count 1: 5 

Optimal solution found (tolerance 1.00e-04)
Best objective


Optimize a model with 29340 rows, 15860 columns and 67008 nonzeros
Model fingerprint: 0x0a69faa4
Variable types: 0 continuous, 15860 integer (15860 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 29340 rows and 15860 columns
Presolve time: 0.28s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.30 seconds (0.27 work units)
Thread count was 1 (of 72 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%
IP Starts
Set parameter TimeLimit to value 30
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU E5-2697 v4 @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 36 physical cores, 72 logical processors, using up to 32 threads

Optimize a model with 29340 rows, 158

...

Model fingerprint: 0xf3124147
Variable types: 0 continuous, 19152 integer (19152 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 33455 rows and 19152 columns
Presolve time: 0.55s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.57 seconds (0.42 work units)
Thread count was 1 (of 72 available processors)

Solution count 1: 5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%
IP Starts
Set parameter TimeLimit to value 30
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU E5-2697 v4 @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 36 physical cores, 72 logical processors, using up to 32 threads

Optimize a model with 33455 rows, 19152 columns and 75035 nonzeros
Model fingerprint: 0x744d91a0
Variable


Solution count 1: 5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%
Galactose
IP Starts
Set parameter TimeLimit to value 30
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU E5-2697 v4 @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 36 physical cores, 72 logical processors, using up to 32 threads

Optimize a model with 32616 rows, 18626 columns and 72912 nonzeros
Model fingerprint: 0x3ea26c41
Variable types: 0 continuous, 18626 integer (18626 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 29790 rows and 17939 columns
Presolve time: 1.06s
Presolved: 2826 rows, 687 columns, 7007 nonzeros
Variable types: 0 continuous, 687 integer (687 binary)
Found heuristic solution: objective 3.0000000
Found heuristic solution: objective 1.00

Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 30071 rows and 17945 columns
Presolve time: 1.19s
Presolved: 2550 rows, 681 columns, 6181 nonzeros
Variable types: 0 continuous, 681 integer (681 binary)
Found heuristic solution: objective 2.0000000

Explored 0 nodes (0 simplex iterations) in 1.23 seconds (0.66 work units)
Thread count was 32 (of 72 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%
FAULT:  9.0 2.0 9
IP Starts
Set parameter TimeLimit to value 30
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU E5-2697 v4 @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 36 physical cores, 72 logical processors, using up to 32 threads

Optimize a model with 32621 rows, 18626 columns and 72917 nonzer

Presolve time: 0.20s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.23 seconds (0.22 work units)
Thread count was 1 (of 72 available processors)

Solution count 1: 5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%
FAULT:  8.0 5.0 8
IP Starts
Set parameter TimeLimit to value 30
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU E5-2697 v4 @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 36 physical cores, 72 logical processors, using up to 32 threads

Optimize a model with 32631 rows, 18626 columns and 72927 nonzeros
Model fingerprint: 0xbd9a59d3
Variable types: 0 continuous, 18626 integer (18626 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 32631 rows and 18626 columns
Presolve time: 0.21s

Thread count: 36 physical cores, 72 logical processors, using up to 32 threads

Optimize a model with 32636 rows, 18626 columns and 72932 nonzeros
Model fingerprint: 0xf6c42bba
Variable types: 0 continuous, 18626 integer (18626 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 32636 rows and 18626 columns
Presolve time: 0.20s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.22 seconds (0.21 work units)
Thread count was 1 (of 72 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [24]:
selection_enzy

[['2.3.1.15'], ['2.7.1.94'], ['1.2.1.3']]

In [16]:
solution['FE[0,17]']

0.0

In [5]:
import networkx as nx
G = nx.DiGraph()
G.add_edges_from(edges)

idx_reac = 0
idx_comp = 0
for node in G.nodes:
    if node in list_reactions:
        G.nodes[node]['index'] = idx_reac
        idx_reac = idx_reac + 1
    else:
        G.nodes[node]['index'] = idx_comp
        idx_comp = idx_comp + 1

In [8]:
target_list = ['C00681',
'C00116']

In [2]:
def gurobi_solve(list_reactions, list_compounds, target_list, edges, G):
    print("IP Starts")
    #print(list_compounds)
    #print(edges)
    #break
    IP_time_limit = 30
    prob = Model("CutRec")
    prob.setParam('TimeLimit', IP_time_limit)
    sum_ = (len(edges) + len(list_reactions) + len(list_compounds))
    FE = prob.addVars(sum_+2, len(list_reactions),vtype=GRB.BINARY, name="FE")
    TE = prob.addVars(sum_+2, len(list_reactions),vtype=GRB.BINARY, name="TE")

    FR = prob.addVars(sum_+2, len(list_reactions),vtype=GRB.BINARY, name="FR")
    TR = prob.addVars(sum_+2, len(list_reactions),vtype=GRB.BINARY, name="TR")

    FC = prob.addVars(sum_+2, len(list_compounds),vtype=GRB.BINARY, name="FC")
    TC = prob.addVars(sum_+2, len(list_compounds),vtype=GRB.BINARY, name="TC")

    nontarget_list = []
    for comp in list_compounds:
        if comp not in target_list:
            nontarget_list.append(comp)

    #prob.setObjective(quicksum(FE[0,G.nodes[node]['index']] for node in list_reactions), GRB.MINIMIZE)
    prob.setObjective(quicksum(FC[sum_,G.nodes[node]['index']] for node in nontarget_list), GRB.MINIMIZE)

    for node in target_list:
        comp_idx = G.nodes[node]['index']
        #print(sum_,comp_idx)
        prob.addConstr(FC[sum_,comp_idx] == 1)

    for t in range(0, sum_+1):
        for reac_node in list_reactions:
            reac_idx = G.nodes[reac_node]['index']

            if t == 0:
                prob.addConstr(TR[t,reac_idx] == 1)

            prob.addConstr(TR[t,reac_idx] + FR[t,reac_idx] == 1)
            prob.addConstr(TE[t,reac_idx] + FE[t,reac_idx] == 1)

            prob.addConstr(FR[t+1,reac_idx] + TE[t,reac_idx] >= 1)
            prob.addConstr(TR[t+1,reac_idx] + FE[t,reac_idx] + quicksum(FC[t,G.nodes[node]['index']] for node in G.predecessors(reac_node)) >= 1)

            for node in G.predecessors(reac_node):
                prob.addConstr(FR[t+1,reac_idx] + TC[t,G.nodes[node]['index']] >= 1)

        for comp_node in list_compounds:
            comp_idx = G.nodes[comp_node]['index']

            prob.addConstr(TC[t,comp_idx] + FC[t,comp_idx] == 1)

            if len(list(G.predecessors(comp_node))) > 0:
                if t == 0:
                    prob.addConstr(TC[t,comp_idx] == 1)
                #print(comp_node)
                prob.addConstr(FC[t+1,comp_idx] + quicksum(TR[t,G.nodes[node]['index']] for node in G.predecessors(comp_node)) >= 1)
                for node in G.predecessors(comp_node):
                    prob.addConstr(TC[t+1,comp_idx] + FR[t,G.nodes[node]['index']] >= 1)
            else:
                prob.addConstr(TC[t+1,comp_idx] == 1)

        for reac_node in list_reactions:
            reac_idx = G.nodes[reac_node]['index']
            prob.addConstr(TE[t+1,reac_idx] == TE[t,reac_idx])
    prob.optimize()
    solution = {}
    for v in prob.getVars():
        #print('%s %g' % (v.VarName, v.X))
        solution[v.VarName] = v.X
    return prob.ObjBound, prob.runtime, solution


In [7]:
prob

<gurobi.Model Continuous instance CutRec: 0 constrs, 0 vars, Parameter changes: TimeLimit=30.0, Username=(user-defined)>

In [10]:
prob.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU E5-2697 v4 @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 36 physical cores, 72 logical processors, using up to 32 threads

Optimize a model with 4433 rows, 2944 columns and 9428 nonzeros
Model fingerprint: 0x804bf0e3
Variable types: 0 continuous, 2944 integer (2944 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4433 rows and 2944 columns
Presolve time: 0.03s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.01 work units)
Thread count was 1 (of 72 available processors)

Solution count 1: 4 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+00, best bound 4.000000000000e+00, gap 0.0000%


In [23]:
prob.runtime

0.04631614685058594

In [11]:
for v in prob.getVars():
    print('%s %g' % (v.VarName, v.X))

FE[0,0] 0
FE[0,1] 1
FE[0,2] 0
FE[0,3] 1
FE[0,4] 1
FE[0,5] 0
FE[0,6] 0
FE[0,7] 0
FE[0,8] 0
FE[0,9] 1
FE[0,10] 0
FE[1,0] 0
FE[1,1] 1
FE[1,2] 0
FE[1,3] 1
FE[1,4] 1
FE[1,5] 0
FE[1,6] 0
FE[1,7] 0
FE[1,8] 0
FE[1,9] 1
FE[1,10] 0
FE[2,0] 0
FE[2,1] 1
FE[2,2] 0
FE[2,3] 1
FE[2,4] 1
FE[2,5] 0
FE[2,6] 0
FE[2,7] 0
FE[2,8] 0
FE[2,9] 1
FE[2,10] 0
FE[3,0] 0
FE[3,1] 1
FE[3,2] 0
FE[3,3] 1
FE[3,4] 1
FE[3,5] 0
FE[3,6] 0
FE[3,7] 0
FE[3,8] 0
FE[3,9] 1
FE[3,10] 0
FE[4,0] 0
FE[4,1] 1
FE[4,2] 0
FE[4,3] 1
FE[4,4] 1
FE[4,5] 0
FE[4,6] 0
FE[4,7] 0
FE[4,8] 0
FE[4,9] 1
FE[4,10] 0
FE[5,0] 0
FE[5,1] 1
FE[5,2] 0
FE[5,3] 1
FE[5,4] 1
FE[5,5] 0
FE[5,6] 0
FE[5,7] 0
FE[5,8] 0
FE[5,9] 1
FE[5,10] 0
FE[6,0] 0
FE[6,1] 1
FE[6,2] 0
FE[6,3] 1
FE[6,4] 1
FE[6,5] 0
FE[6,6] 0
FE[6,7] 0
FE[6,8] 0
FE[6,9] 1
FE[6,10] 0
FE[7,0] 0
FE[7,1] 1
FE[7,2] 0
FE[7,3] 1
FE[7,4] 1
FE[7,5] 0
FE[7,6] 0
FE[7,7] 0
FE[7,8] 0
FE[7,9] 1
FE[7,10] 0
FE[8,0] 0
FE[8,1] 1
FE[8,2] 0
FE[8,3] 1
FE[8,4] 1
FE[8,5] 0
FE[8,6] 0
FE[8,7] 0
FE[8,8] 0
FE[8,9] 1
FE[8,10] 0
F

TC[28,9] 1
TC[29,0] 0
TC[29,1] 1
TC[29,2] 0
TC[29,3] 0
TC[29,4] 0
TC[29,5] 0
TC[29,6] 1
TC[29,7] 1
TC[29,8] 0
TC[29,9] 1
TC[30,0] 0
TC[30,1] 1
TC[30,2] 0
TC[30,3] 0
TC[30,4] 0
TC[30,5] 0
TC[30,6] 1
TC[30,7] 1
TC[30,8] 0
TC[30,9] 1
TC[31,0] 0
TC[31,1] 1
TC[31,2] 0
TC[31,3] 0
TC[31,4] 0
TC[31,5] 0
TC[31,6] 1
TC[31,7] 1
TC[31,8] 0
TC[31,9] 1
TC[32,0] 0
TC[32,1] 1
TC[32,2] 0
TC[32,3] 0
TC[32,4] 0
TC[32,5] 0
TC[32,6] 1
TC[32,7] 1
TC[32,8] 0
TC[32,9] 1
TC[33,0] 0
TC[33,1] 1
TC[33,2] 0
TC[33,3] 0
TC[33,4] 0
TC[33,5] 0
TC[33,6] 1
TC[33,7] 1
TC[33,8] 0
TC[33,9] 1
TC[34,0] 0
TC[34,1] 1
TC[34,2] 0
TC[34,3] 0
TC[34,4] 0
TC[34,5] 0
TC[34,6] 1
TC[34,7] 1
TC[34,8] 0
TC[34,9] 1
TC[35,0] 0
TC[35,1] 1
TC[35,2] 0
TC[35,3] 0
TC[35,4] 0
TC[35,5] 0
TC[35,6] 1
TC[35,7] 1
TC[35,8] 0
TC[35,9] 1
TC[36,0] 0
TC[36,1] 1
TC[36,2] 0
TC[36,3] 0
TC[36,4] 0
TC[36,5] 0
TC[36,6] 1
TC[36,7] 1
TC[36,8] 0
TC[36,9] 1
TC[37,0] 0
TC[37,1] 1
TC[37,2] 0
TC[37,3] 0
TC[37,4] 0
TC[37,5] 0
TC[37,6] 1
TC[37,7] 1
TC[37,8] 0
TC[37,9] 1

In [4]:
IP_time_limit = 30
prob = Model("CutRec")
prob.setParam('TimeLimit', IP_time_limit)
sum_ = (len(edges) + len(list_reactions) + len(list_compounds))
FE = prob.addVars(sum_+2, len(list_reactions),vtype=GRB.BINARY, name="FE")
TE = prob.addVars(sum_+2, len(list_reactions),vtype=GRB.BINARY, name="TE")

FR = prob.addVars(sum_+2, len(list_reactions),vtype=GRB.BINARY, name="FR")
TR = prob.addVars(sum_+2, len(list_reactions),vtype=GRB.BINARY, name="TR")

FC = prob.addVars(sum_+2, len(list_compounds),vtype=GRB.BINARY, name="FC")
TC = prob.addVars(sum_+2, len(list_compounds),vtype=GRB.BINARY, name="TC")

nontarget_list = []
for comp in list_compounds:
    if comp not in target_list:
        nontarget_list.append(comp)

#prob.setObjective(quicksum(FE[0,G.nodes[node]['index']] for node in list_reactions), GRB.MINIMIZE)
prob.setObjective(quicksum(FC[sum_,G.nodes[node]['index']] for node in nontarget_list), GRB.MINIMIZE)

for node in target_list:
    comp_idx = G.nodes[node]['index']
    #print(sum_,comp_idx)
    prob.addConstr(FC[sum_,comp_idx] == 1)

for t in range(0, sum_+1):
    for reac_node in list_reactions:
        reac_idx = G.nodes[reac_node]['index']

        if t == 0:
            prob.addConstr(TR[t,reac_idx] == 1)

        prob.addConstr(TR[t,reac_idx] + FR[t,reac_idx] == 1)
        prob.addConstr(TE[t,reac_idx] + FE[t,reac_idx] == 1)

        prob.addConstr(FR[t+1,reac_idx] + TE[t,reac_idx] >= 1)
        prob.addConstr(TR[t+1,reac_idx] + FE[t,reac_idx] + quicksum(FC[t,G.nodes[node]['index']] for node in G.predecessors(reac_node)) >= 1)

        for node in G.predecessors(reac_node):
            prob.addConstr(FR[t+1,reac_idx] + TC[t,G.nodes[node]['index']] >= 1)

    for comp_node in list_compounds:
        comp_idx = G.nodes[comp_node]['index']

        prob.addConstr(TC[t,comp_idx] + FC[t,comp_idx] == 1)

        if len(list(G.predecessors(comp_node))) > 0:
            if t == 0:
                prob.addConstr(TC[t,comp_idx] == 1)
            #print(comp_node)
            prob.addConstr(FC[t+1,comp_idx] + quicksum(TR[t,G.nodes[node]['index']] for node in G.predecessors(comp_node)) >= 1)
            for node in G.predecessors(comp_node):
                prob.addConstr(TC[t+1,comp_idx] + FR[t,G.nodes[node]['index']] >= 1)
        else:
            prob.addConstr(TC[t+1,comp_idx] == 1)

    for reac_node in list_reactions:
        reac_idx = G.nodes[reac_node]['index']
        prob.addConstr(TE[t+1,reac_idx] == TE[t,reac_idx])
    prob.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-15
Set parameter TimeLimit to value 30


NameError: name 'target_list' is not defined

In [3]:
def bfs(state, target, G_metabolic):
  check = {}
  for comp in G_metabolic.nodes:
    check[comp] = True
  queue = []
  for i in range(0, len(list_enzymes)):
    if state[i] == 1:
      queue.append(list_enzymes[i])
      check[list_enzymes[i]] = False
  
  while queue:
    head = queue.pop(0)
    list_successor = [i for i in G_metabolic.neighbors(head)]
    for comp in list_successor:
      if check[comp]:
        if comp in list_compounds:
          list_predecessor = [i for i in G_metabolic.predecessors(comp)]
          double_check = True
          for comp_pre in list_predecessor:
            if check[comp_pre]:
              double_check = False
              break
          if double_check:
            check[comp] = False
            queue.append(comp)
        else:
          check[comp] = False
          queue.append(comp)
  damage = 0
  removal = 0
  for comp in list_compounds:
    if comp in target:
      removal = removal + (check[comp] == False)
      #if check[comp]:
      #  print(comp)
    else:
      #if not check[comp]:
      #  print(comp)
      damage = damage + (check[comp] == False)
      
  return damage, removal == len(target)
